# Restitution

**Definition:**
> The coefficient of restitution (COR), also denoted by ($c_r$), is the ratio of the final to initial relative velocity between two objects after they collide. It normally ranges from 0 to 1 where 1 would be a perfectly elastic collision. A perfectly inelastic collision has a coefficient of 0, but a 0 value does not have to be perfectly inelastic.

Our system already exhibits restitutive properties. However, the restitution is not directly controllable and depends on a variety of parameters (e.g., $h$, $\hat{d}$, and $\kappa$) and the integration scheme. 

Let us start by examining the restitutive properties of our formulation using a one dimensional example. A particle ($x_0=10$) is falling towards a ground plane ($x=0$). To handle collisions we use our IPC barrier directly on the $x$ position.

In [ ]:
import numpy
import plotly.graph_objects as go

We will start by defining the implicit Euler time-integration scheme in variational form
\begin{equation}
E(x) = \frac{1}{2}m(x-\hat{x})^2,
\end{equation}
where $\hat{x}$ for implicit Euler is 
\begin{equation}
\hat{x} = x^t + hv^t + h^2g,
\end{equation}
and for implicit Newmark ($\gamma=\frac{1}{2}$, $\beta = \frac{1}{4}$) is
\begin{equation}
\hat{x} = x^t + hv^t + h^2g + \frac{h^2}{2}m^{-1}V(x_t),
\end{equation}
The next value of $x$ is then given as 
\begin{equation}
x^{t+1} = \text{arg}\min_x E(x),
\end{equation}
and we using an explicit update for the velocity
\begin{equation}
v^{t+1} = \frac{x^{t+1} - x^t}{h}.
\end{equation}

In [ ]:
def euler_update(x_t, v_t, g, h, m):
    """Euler update for (γ=1/2 and β=1/4)."""
    return x_t + h * (v_t + h * g)


def newmark_update(x_t, v_t, g, h, m, V):
    """Newmark update for (γ=1/2 and β=1/4)."""
    return x_t + h * (v_t + h * (g + 0.5 * V(x_t)))


def E(x, m, xhat):
    return 0.5 * m * (x - xhat)**2


# We also need the derivative of E in order to minimize Equation E(x)
def dE(x, m, xhat):
    return m * (x - xhat)


def ddE(x, m, xhat):
    return m

To handle collision we will use the IPC barrier
$$
B(x) = -\ln\left(\frac{x}{\hat{d}}\right)(x-\hat{d})^2.
$$
Our variational form integration then becomes
$$
x^{t+1} = \text{arg}\min_x E(x)+\kappa B(x),
$$
where $\kappa$ is our barrier stiffness.

In [ ]:
def B(x, dhat):
    if x >= dhat:
        return 0
    if x <= 0:
        return numpy.infty
    return -numpy.log(x / dhat) * (x - dhat)**2


def dB(x, dhat):
    if x <= 0 or x >= dhat:
        return 0
    return (dhat - x) * (2 * numpy.log(x / dhat) - dhat / x + 1)


def ddB(x, dhat):
    if x <= 0 or x >= dhat:
        return 0
    dhat_over_x = dhat / x
    return (dhat_over_x + 2) * dhat_over_x - 2 * numpy.log(x / dhat) - 3

We minimize our energy using a simple Newton method with line-search.

In [ ]:
def newton(f, df, ddf, x0, h, conv_tol=1e-5):
    x = x0
    for i in range(100):
        fx = f(x)
        dfx = df(x)
        ddfx = ddf(x)
        delta_x = -dfx / ddfx

        if abs(delta_x) / h < conv_tol:
            # print(f"found minimum (delta_x={delta_x:g})")
            break

        step_size = 1
        # Line search:
        # No need to check for x < 0 because x is signed and B(x) = infinity if x < 0.
        while step_size > 0 and f(x + step_size * delta_x) >= fx:
            step_size /= 2

        if step_size == 0:
            print("line-search failed")
            break

        x += step_size * delta_x
    return x

In [ ]:
def sim(m=1,
        x_t=10,
        v_t=-10,
        g=0,
        h=1e-3,
        dhat=1e-3,
        kappa=100,
        conv_tol=1e-5,
        use_newmark=False):
    t = 0
    xs = [x_t]
    vs = [v_t]
    while t < 9.999:
        if use_newmark:
            xhat = newmark_update(x_t, v_t, g, h, m, lambda x: B(x, dhat))
        else:
            xhat = euler_update(x_t, v_t, g, h, m)
        x = newton(lambda x: E(x, m, xhat) + kappa * B(x, dhat),
                   lambda x: dE(x, m, xhat) + kappa * dB(x, dhat),
                   lambda x: ddE(x, m, xhat) + kappa * ddB(x, dhat), x_t, h,
                   conv_tol)
        v_t = (x - x_t) / h
        x_t = x
        t += h
        xs.append(x)
        vs.append(v_t)
    return xs, vs

In [ ]:
x_data = []
v_data = []

for use_newmark in False, True:
    xs, vs = sim(h=1e-4,
                 dhat=1e-3,
                 kappa=100,
                 conv_tol=1e-5,
                 use_newmark=use_newmark)
    ts = numpy.arange(len(xs)) / (len(xs) - 1) * 10
    x_data.append(
        go.Scatter(x=ts, y=xs, name=("Newmark" if use_newmark else "Euler")))
    v_data.append(
        go.Scatter(x=ts, y=vs, name=("Newmark" if use_newmark else "Euler")))

fig = go.Figure(data=x_data)
fig.update_layout(
    title=
    "Constant Velocity (Position) (h=1e-4, dhat=1e-3, kappa=100, conv_tol=1e-5)",
    xaxis_title="t")
fig.show()
fig = go.Figure(data=v_data)
fig.update_layout(
    title=
    "Constant Velocity (Velocity) (h=1e-4, dhat=1e-3, kappa=100, conv_tol=1e-5)",
    xaxis_title="t")
fig.show()

In [ ]:
xs, vs = sim(g=-9.8, h=1e-4, dhat=1e-3, kappa=100, conv_tol=1e-5)
ts = numpy.arange(len(xs)) / (len(xs) - 1) * 10
fig = go.Figure(data=[go.Scatter(x=ts, y=xs, name="height")])
fig.update_layout(
    title="Under Gravity (h=1e-4, dhat=1e-3, kappa=100, conv_tol=1e-5)",
    xaxis_title="t",
    yaxis_title="height")
fig.show()

In [ ]:
data = []
for dhat in numpy.logspace(0, -6, 7):
    xs, _ = sim(h=1e-3, dhat=dhat, kappa=100, conv_tol=1e-5)
    ts = numpy.arange(len(xs)) / (len(xs) - 1) * 10
    data.append(go.Scatter(x=ts, y=xs, name=f"dhat={dhat:g}"))
fig = go.Figure(data=data)
fig.update_layout(title="Varying dhat (h=1e-3, kappa=100, conv_tol=1e-5)",
                  xaxis_title="t",
                  yaxis_title="height")
fig.show()

In [ ]:
data = []
for h in numpy.logspace(0, -5, 6):
    xs, _ = sim(h=h, dhat=1e-3, kappa=100, conv_tol=1e-5)
    ts = numpy.arange(len(xs)) / (len(xs) - 1) * 10
    data.append(go.Scatter(x=ts, y=xs, name=f"h={h:g}"))
fig = go.Figure(data=data)
fig.update_layout(title="Varying h (dhat=1e-3, kappa=100, conv_tol=1e-5)",
                  xaxis_title="t",
                  yaxis_title="height")
fig.show()

In [ ]:
data = []
for kappa in numpy.logspace(0, 8, 9):
    xs, _ = sim(h=1e-3, dhat=1e-3, kappa=kappa, conv_tol=1e-5)
    ts = numpy.arange(len(xs)) / (len(xs) - 1) * 10
    data.append(go.Scatter(x=ts, y=xs, name=f"kappa={kappa:g}"))
fig = go.Figure(data=data)
fig.update_layout(title="Varying kappa (h=1e-3, dhat=1e-3, conv_tol=1e-5)",
                  xaxis_title="t",
                  yaxis_title="height")
fig.show()

In [ ]:
data = []
for conv_tol in numpy.logspace(-1, -10, 10):
    xs, _ = sim(h=1e-3, dhat=1e-3, kappa=100, conv_tol=conv_tol)
    ts = numpy.arange(len(xs)) / (len(xs) - 1) * 10
    data.append(go.Scatter(x=ts, y=xs, name=f"conv_tol={conv_tol:g}"))
fig = go.Figure(data=data)
fig.update_layout(title="Varying conv_tol (h=1e-3, dhat=1e-3, kappa=100)",
                  xaxis_title="t",
                  yaxis_title="height")
fig.show()

In [ ]:
data = []
for v0 in numpy.linspace(-2, -50, 10):
    xs, vs = sim(v_t=v0, h=1e-3, dhat=1e-3, kappa=100, conv_tol=1e-5)
    ts = numpy.arange(len(xs)) / (len(xs) - 1) * 10
    data.append(go.Scatter(x=ts, y=abs((numpy.array(vs) / v0)), name=f"v0={v0:g}"))
fig = go.Figure(data=data)
fig.update_layout(title="Varying v0 (h=1e-3, dhat=1e-3, kappa=100, conv_tol=1e-5)",
                  xaxis_title="t",
                  yaxis_title="v / v0")
fig.show()